<a href="https://colab.research.google.com/github/JonathanLehner/Colab-collection/blob/master/manopth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# code to run a demo of https://github.com/hassony2/manopth



```
# This is formatted as code
```



In [3]:
!git clone https://github.com/hassony2/manopth

Cloning into 'manopth'...
remote: Enumerating objects: 48, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 48 (delta 14), reused 44 (delta 12), pack-reused 0
Unpacking objects: 100% (48/48), done.


In [4]:
% cd /content/manopth

/content/manopth


In [0]:
# install conda
# see https://facebookresearch.github.io/TensorComprehensions/installation_colab_research.html
!wget -c https://repo.continuum.io/archive/Anaconda3-5.1.0-Linux-x86_64.sh
!chmod +x Anaconda3-5.1.0-Linux-x86_64.sh
!bash ./Anaconda3-5.1.0-Linux-x86_64.sh -b -f -p /usr/local
#!conda install -q -y --prefix /usr/local -c pytorch
!conda env create -f environment.yml

import sys
sys.path.append('/usr/local/lib/python3.6/site-packages/')


--2019-04-30 16:28:25--  https://repo.continuum.io/archive/Anaconda3-5.1.0-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.201.79, 104.18.200.79, 2606:4700::6812:c84f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.201.79|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 577996269 (551M) [application/x-sh]
Saving to: ‘Anaconda3-5.1.0-Linux-x86_64.sh’

Anaconda3-5.1.0-Lin 100%[===================>] 551.22M  51.8MB/s    in 11s     

2019-04-30 16:28:36 (52.2 MB/s) - ‘Anaconda3-5.1.0-Linux-x86_64.sh’ saved [577996269/577996269]

PREFIX=/usr/local
installing: python-3.6.4-hc3d631a_1 ...
Python 3.6.4 :: Anaconda, Inc.
installing: ca-certificates-2017.08.26-h1d4fec5_0 ...
installing: conda-env-2.6.0-h36134e3_1 ...
installing: intel-openmp-2018.0.0-hc7b2577_8 ...
installing: libgcc-ng-7.2.0-h7cc24e2_2 ...
installing: libgfortran-ng-7.2.0-h9f7466a_2 ...
installing: libstdcxx-ng-7.2.0-h7a57d05_2 ...
installing: bzip2-1.0.6-h9a117

In [0]:
!nvcc --version
!conda install pytorch torchvision cudatoolkit=10.0 -c pytorch


In [0]:
!wget https://github.com/JonathanLehner/Colab-collection/releases/download/MANO/mano_v1_2.zip
!unzip mano_v1_2.zip
!mv /content/manopth/mano_v1_2/models /content/manopth/mano/models 

In [0]:
!source activate manopth
!pip install git+https://github.com/hassony2/chumpy.git
!pip install opencv-python
!pip install tqdm

In [0]:
%cd /content/manopth/
#!pip3 install --upgrade setuptools
#!apt-get install python3.7-dev libmysqlclient-dev
#!pip3 install .
#sudo apt-get install libmysqlclient-dev

!python setup.py install

In [0]:
!conda update -n base conda
!conda list pytorch

In [0]:
command_string = "python /content/manopth/examples/manopth_mindemo.py"

import subprocess
import shlex
args = shlex.split(command_string)
print(args)
process = subprocess.Popen(args, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, bufsize=1)

while True:
    output = process.stdout.readline()
    if process.poll() is not None:
        break
    if output != b'':
        print(output)
    rc = process.poll()


In [0]:
#To check that everything is going well, run python examples/manopth_mindemo.py, 
#which should generate from a random hand using the MANO layer !
%cd /content/manopth/
!python /content/manopth/examples/manopth_mindemo.py

In [0]:
# have to inline /content/manopth/examples/manopth_mindemo.py
import torch
from manopth.manolayer import ManoLayer
from manopth import demo

batch_size = 10
# Select number of principal components for pose space
ncomps = 6

# Initialize MANO layer
mano_layer = ManoLayer(
    mano_root='mano/models', use_pca=True, ncomps=ncomps, flat_hand_mean=False)

# Generate random shape parameters
random_shape = torch.rand(batch_size, 10)
# Generate random pose parameters, including 3 values for global axis-angle rotation
random_pose = torch.rand(batch_size, ncomps + 3)

# Forward pass through MANO layer
hand_verts, hand_joints = mano_layer(random_pose, random_shape)
demo.display_hand({
    'verts': hand_verts,
    'joints': hand_joints
},
mano_faces=mano_layer.th_faces)

# source: https://stackoverflow.com/questions/34495817/plotting-a-background-image-to-a-matplotlib-graph
from matplotlib import pyplot as plt
from scipy.misc import face
import numpy as np

fig, ax = plt.subplots(1, 1)

x = np.linspace(0, 1, 10)
y = np.random.rand(10)
image = ax.imshow(face(), cmap=plt.cm.gray, extent=[0, 1, 0, 1])
line = ax.plot(x, y, '-r', lw=2)

# invisible clipping polygon
poly = ax.fill_between(x, 0, y, visible=False)
clip = poly.get_paths()[0]

# you will need to do this separately for each of the images in your plot
image.set_clip_path(clip, transform=ax.transData)

plt.show()


In [0]:
# from https://github.com/hassony2/manopth/blob/master/manopth/demo.py
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from manopth import demo

def display_hand(hand_info, mano_faces=None, ax=None, alpha=0.2, batch_idx=0, show=True):
    """
    Displays hand batch_idx in batch of hand_info, hand_info as returned by
    generate_random_hand
    """
    if ax is None:
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
    verts, joints = hand_info['verts'][batch_idx], hand_info['joints'][
        batch_idx]
    if mano_faces is None:
        ax.scatter(verts[:, 0], verts[:, 1], verts[:, 2], alpha=0.1)
    else:
        mesh = Poly3DCollection(verts[mano_faces], alpha=alpha)
        face_color = (141 / 255, 184 / 255, 226 / 255)
        edge_color = (50 / 255, 50 / 255, 50 / 255)
        mesh.set_edgecolor(edge_color)
        mesh.set_facecolor(face_color)
        ax.add_collection3d(mesh)
    ax.scatter(joints[:, 0], joints[:, 1], joints[:, 2], color='r')
    
    # add background image
    image = ax.imshow(face(), cmap=plt.cm.gray, extent=[0, -0.1, 0, 0.1])
    
    demo.cam_equal_aspect_3d(ax, verts.numpy())
    if show:
        plt.show()

# Forward pass through MANO layer
hand_verts, hand_joints = mano_layer(random_pose, random_shape)
display_hand({
    'verts': hand_verts,
    'joints': hand_joints
},
mano_faces=mano_layer.th_faces)

In [0]:
#https://stackoverflow.com/questions/34458251/plot-over-an-image-background-in-python/34459284

In [0]:
#To be able to import and use ManoLayer in another project, go to your manopth folder and run 
#pip install . cd /path/to/other/project